In [46]:
import pandas as pd
pd.set_option('display.float_format', '{:.5f}'.format)
import numpy as np

In [47]:
ticker="spy"
df=pd.read_csv(f"data/raw/{ticker}_20250103_20250131.csv",engine="pyarrow", sep = ',')
df["Date-Time"] = df["Date-Time"].dt.tz_convert("America/New_York")
df.head(3)
df.describe()

,L1-BidPrice,L1-BidSize,L1-AskPrice,L1-AskSize,L2-BidPrice,L2-BidSize,L2-AskPrice,L2-AskSize,L3-BidPrice,L3-BidSize,...,L23-AskPrice,L23-AskSize,L24-BidPrice,L24-BidSize,L24-AskPrice,L24-AskSize,L25-BidPrice,L25-BidSize,L25-AskPrice,L25-AskSize
count,15854881.00000,15854881.00000,15855702.00000,15855702.00000,15840006.00000,15840006.00000,15847829.00000,15847829.00000,15812279.00000,15812279.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
mean,20.21678,2553.69399,20.22971,2572.70153,20.20669,3021.62034,20.23995,3112.41568,20.19669,3510.96515,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,1.67889,2309.88877,1.67916,2292.27281,1.67942,2010.94648,1.67951,2025.08586,1.68036,1787.92233,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,16.35000,1.00000,16.36000,1.00000,16.34000,1.00000,16.37000,2.00000,16.33000,1.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,18.88000,800.00000,18.89000,800.00000,18.87000,1400.00000,18.90000,1400.00000,18.86000,2500.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,20.40000,1900.00000,20.42000,1900.00000,20.39000,2700.00000,20.43000,2900.00000,20.39000,3300.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,21.68000,3687.00000,21.69000,3700.00000,21.67000,4200.00000,21.70000,4400.00000,21.66000,4600.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,23.45000,25826.00000,23.51000,24812.00000,23.44000,25826.00000,23.57000,24300.00000,23.43000,14900.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
pd.DataFrame(pd.to_datetime(df["Date-Time"]).dt.date.unique())

,0
0,2025-01-03
1,2025-01-06
2,2025-01-07
3,2025-01-08
4,2025-01-10
5,2025-01-13
6,2025-01-14
7,2025-01-15
8,2025-01-16
9,2025-01-17


In [49]:
df["Date"] = pd.to_datetime(df["Date-Time"]).dt.date
grouped = df.groupby("Date").size().reset_index(name="Count")


In [50]:
# Filter only regular market hours (09:30–16:00 Eastern)
df_market = df[
    (df["Date-Time"].dt.time >= pd.to_datetime("09:40:00").time()) &
    (df["Date-Time"].dt.time <= pd.to_datetime("15:50:00").time())
]

In [51]:
base_cols = ["#RIC", "Date-Time"]
#base_cols = ["#RIC"]
# Dynamically build list for L1–L10
levels = []
for i in range(1, 11):  # L1–L10 inclusive
    levels += [
        f"L{i}-BidPrice", f"L{i}-BidSize",
        f"L{i}-AskPrice", f"L{i}-AskSize"
    ]


In [52]:
df_market_l10 = df_market[base_cols + levels]

In [53]:
df_market_l10.head(10)
df_market_l10.describe()

,L1-BidPrice,L1-BidSize,L1-AskPrice,L1-AskSize,L2-BidPrice,L2-BidSize,L2-AskPrice,L2-AskSize,L3-BidPrice,L3-BidSize,...,L8-AskPrice,L8-AskSize,L9-BidPrice,L9-BidSize,L9-AskPrice,L9-AskSize,L10-BidPrice,L10-BidSize,L10-AskPrice,L10-AskSize
count,13998179.00000,13998179.00000,13998179.00000,13998179.00000,13998179.00000,13998179.00000,13998179.00000,13998179.00000,13998179.00000,13998179.00000,...,13998179.00000,13998179.00000,13998177.00000,13998177.00000,13998177.00000,13998177.00000,13998174.00000,13998174.00000,13998171.00000,13998171.00000
mean,20.21058,2680.76135,20.22250,2700.13681,20.20056,3180.46700,20.23256,3274.66585,20.19055,3687.98224,...,20.30083,2064.85175,20.13045,1038.42142,20.32572,2097.85417,20.12041,772.95305,20.35950,2558.68570
std,1.69132,2298.18489,1.69156,2294.80126,1.69133,1945.90720,1.69157,1978.52464,1.69133,1661.53360,...,1.69310,1797.21772,1.69128,1374.26794,1.69519,2755.02803,1.69127,1518.66370,1.69940,3338.53773
min,16.35000,1.00000,16.36000,1.00000,16.34000,1.00000,16.37000,3.00000,16.33000,3.00000,...,16.43000,1.00000,16.27000,30.00000,16.44000,1.00000,15.24000,2.00000,16.45000,1.00000
25%,18.86000,900.00000,18.87000,900.00000,18.85000,1600.00000,18.88000,1700.00000,18.84000,2600.00000,...,18.95000,400.00000,18.78000,200.00000,18.96000,200.00000,18.77000,200.00000,18.98000,100.00000
50%,20.35000,2100.00000,20.36000,2100.00000,20.34000,2900.00000,20.37000,3100.00000,20.33000,3500.00000,...,20.44000,2200.00000,20.27000,400.00000,20.48000,1700.00000,20.26000,300.00000,20.51000,2100.00000
75%,21.70000,3800.00000,21.72000,3900.00000,21.69000,4300.00000,21.73000,4500.00000,21.68000,4700.00000,...,21.79000,2500.00000,21.62000,1500.00000,21.81000,2500.00000,21.61000,600.00000,21.84000,2700.00000
max,23.45000,25826.00000,23.51000,24812.00000,23.44000,25826.00000,23.57000,22100.00000,23.43000,14900.00000,...,23.93000,16700.00000,23.37000,15900.00000,25.20000,15600.00000,23.36000,15500.00000,26.02000,15600.00000


In [54]:
price_cols = df_market_l10.filter(regex=r'^L(?:10|[1-9])-(?:Bid|Ask)Price$').columns

# Keep only rows where ANY price column changed vs the previous row
changed = df_market_l10[price_cols].ne(df_market_l10[price_cols].shift()).any(axis=1)
df_filtered = df_market_l10[changed].copy()

In [55]:
print(len(df_filtered))
print(len(df_market_l10))

1162909
13998179


In [56]:
df_market_l10=df_filtered

In [57]:
df_market_l10["ms_timestamp"] = df_market_l10["Date-Time"].dt.floor("ms")
df_market_l10 = (
   df_market_l10.drop_duplicates(subset=["#RIC", "ms_timestamp"], keep="last")
     .drop(columns="ms_timestamp")
)
df_market_l10=df_market_l10.dropna()

In [58]:
df_market_l10.head(10)
df_market_l10.describe()

,L1-BidPrice,L1-BidSize,L1-AskPrice,L1-AskSize,L2-BidPrice,L2-BidSize,L2-AskPrice,L2-AskSize,L3-BidPrice,L3-BidSize,...,L8-AskPrice,L8-AskSize,L9-BidPrice,L9-BidSize,L9-AskPrice,L9-AskSize,L10-BidPrice,L10-BidSize,L10-AskPrice,L10-AskSize
count,956903.00000,956903.00000,956903.00000,956903.00000,956903.00000,956903.00000,956903.00000,956903.00000,956903.00000,956903.00000,...,956903.00000,956903.00000,956903.00000,956903.00000,956903.00000,956903.00000,956903.00000,956903.00000,956903.00000,956903.00000
mean,20.40745,2399.29856,20.42249,2452.23569,20.39739,3162.02579,20.43262,3195.98713,20.38736,3415.14649,...,20.50441,2019.21080,20.32711,931.25248,20.53250,2199.89450,20.31702,716.47434,20.56987,2667.51158
std,1.63231,2222.35413,1.63238,2216.75547,1.63237,2073.98572,1.63241,2026.54017,1.63237,1613.20695,...,1.63401,1929.40852,1.63229,1339.08556,1.63606,2903.76433,1.63228,1504.96639,1.64087,3356.52655
min,16.35000,1.00000,16.36000,1.00000,16.34000,3.00000,16.37000,5.00000,16.33000,3.00000,...,16.43000,1.00000,16.27000,30.00000,16.44000,1.00000,15.24000,2.00000,16.45000,1.00000
25%,19.01000,700.00000,19.02000,700.00000,19.00000,1600.00000,19.03000,1700.00000,18.99000,2500.00000,...,19.10000,300.00000,18.93000,200.00000,19.12000,200.00000,18.92000,100.00000,19.15000,100.00000
50%,20.83000,1800.00000,20.84000,1800.00000,20.82000,2900.00000,20.85000,3000.00000,20.81000,3200.00000,...,20.93000,2100.00000,20.75000,400.00000,20.97000,1700.00000,20.74000,200.00000,21.01000,2300.00000
75%,21.79000,3500.00000,21.80000,3600.00000,21.78000,4200.00000,21.81000,4300.00000,21.77000,4300.00000,...,21.88000,2500.00000,21.71000,900.00000,21.92000,2500.00000,21.70000,500.00000,21.96000,2700.00000
max,23.45000,21611.00000,23.49000,19201.00000,23.44000,25826.00000,23.53000,22054.00000,23.43000,14900.00000,...,23.93000,16700.00000,23.37000,15700.00000,25.20000,15600.00000,23.36000,15500.00000,26.02000,15600.00000


In [59]:
#df_market_l10.drop("mid_price",axis=1, inplace=True)
#df_market_l10.drop("abs.spread",axis=1, inplace=True)
#df_market_l10.drop("rel.spread",axis=1, inplace=True)

In [60]:
df_market_l10.insert(
            2, "mid_price", (df_market_l10["L1-BidPrice"] + df_market_l10["L1-AskPrice"]) / 2
        )  # Add the mid-price column to the orderbook dataframe.
df_market_l10.insert(
            2, "abs.spread", (df_market_l10["L1-AskPrice"] - df_market_l10["L1-BidPrice"])
        )
df_market_l10.insert(
            2, "rel.spread", (df_market_l10["L1-AskPrice"] - df_market_l10["L1-BidPrice"])/df_market_l10["mid_price"]
        )

In [61]:
df_market_l10.head(10)

,#RIC,Date-Time,rel.spread,abs.spread,mid_price,L1-BidPrice,L1-BidSize,L1-AskPrice,L1-AskSize,L2-BidPrice,...,L8-AskPrice,L8-AskSize,L9-BidPrice,L9-BidSize,L9-AskPrice,L9-AskSize,L10-BidPrice,L10-BidSize,L10-AskPrice,L10-AskSize
36748,SOXS.K,2025-01-03 09:40:00.006514971-05:00,0.00094,0.02000,21.30000,21.29000,800.00000,21.31000,2600.00000,21.28000,...,21.38000,1900.00000,21.21000,100.00000,21.43000,100.00000,21.20000,100.00000,21.44000,2300.00000
36749,SOXS.K,2025-01-03 09:40:00.016467552-05:00,0.00047,0.01000,21.30500,21.30000,600.00000,21.31000,2600.00000,21.29000,...,21.38000,1900.00000,21.22000,100.00000,21.43000,100.00000,21.21000,100.00000,21.44000,2300.00000
36758,SOXS.K,2025-01-03 09:40:00.076256984-05:00,0.00094,0.02000,21.31000,21.30000,1100.00000,21.32000,1300.00000,21.29000,...,21.43000,100.00000,21.22000,100.00000,21.44000,2300.00000,21.21000,100.00000,21.52000,2300.00000
36762,SOXS.K,2025-01-03 09:40:00.081235935-05:00,0.00047,0.01000,21.31500,21.31000,600.00000,21.32000,1300.00000,21.30000,...,21.43000,100.00000,21.23000,2000.00000,21.44000,2300.00000,21.22000,100.00000,21.52000,2300.00000
36777,SOXS.K,2025-01-03 09:40:00.116134193-05:00,0.00094,0.02000,21.31000,21.30000,800.00000,21.32000,3900.00000,21.29000,...,21.43000,100.00000,21.22000,100.00000,21.44000,2300.00000,21.21000,100.00000,21.52000,2300.00000
36784,SOXS.K,2025-01-03 09:40:00.126062523-05:00,0.00047,0.01000,21.31500,21.31000,600.00000,21.32000,3900.00000,21.30000,...,21.43000,100.00000,21.23000,2000.00000,21.44000,2300.00000,21.22000,100.00000,21.52000,2300.00000
36789,SOXS.K,2025-01-03 09:40:00.131044286-05:00,0.00094,0.02000,21.31000,21.30000,700.00000,21.32000,3900.00000,21.29000,...,21.43000,100.00000,21.22000,100.00000,21.44000,2300.00000,21.21000,100.00000,21.52000,2300.00000
36791,SOXS.K,2025-01-03 09:40:00.156879201-05:00,0.00047,0.01000,21.30500,21.30000,1900.00000,21.31000,1200.00000,21.29000,...,21.38000,1900.00000,21.22000,100.00000,21.43000,100.00000,21.21000,100.00000,21.44000,2300.00000
36814,SOXS.K,2025-01-03 09:40:00.241564103-05:00,0.00094,0.02000,21.31000,21.30000,1000.00000,21.32000,3200.00000,21.29000,...,21.43000,100.00000,21.22000,100.00000,21.44000,2300.00000,21.21000,100.00000,21.52000,2300.00000
36821,SOXS.K,2025-01-03 09:40:00.346022423-05:00,0.00047,0.01000,21.30500,21.30000,1000.00000,21.31000,600.00000,21.29000,...,21.38000,1900.00000,21.22000,100.00000,21.43000,100.00000,21.21000,100.00000,21.44000,2300.00000


In [62]:
print(sum(round(df_market_l10["abs.spread"],2)==float(0.01)))
tick_size=sum(round(df_market_l10["abs.spread"],2)==float(0.01))/df_market_l10.size
print(tick_size)

515046
0.011960947626527106


In [63]:
df_market_l10.describe()

,rel.spread,abs.spread,mid_price,L1-BidPrice,L1-BidSize,L1-AskPrice,L1-AskSize,L2-BidPrice,L2-BidSize,L2-AskPrice,...,L8-AskPrice,L8-AskSize,L9-BidPrice,L9-BidSize,L9-AskPrice,L9-AskSize,L10-BidPrice,L10-BidSize,L10-AskPrice,L10-AskSize
count,956903.00000,956903.00000,956903.00000,956903.00000,956903.00000,956903.00000,956903.00000,956903.00000,956903.00000,956903.00000,...,956903.00000,956903.00000,956903.00000,956903.00000,956903.00000,956903.00000,956903.00000,956903.00000,956903.00000,956903.00000
mean,0.00074,0.01505,20.41497,20.40745,2399.29856,20.42249,2452.23569,20.39739,3162.02579,20.43262,...,20.50441,2019.21080,20.32711,931.25248,20.53250,2199.89450,20.31702,716.47434,20.56987,2667.51158
std,0.00041,0.00831,1.63234,1.63231,2222.35413,1.63238,2216.75547,1.63237,2073.98572,1.63241,...,1.63401,1929.40852,1.63229,1339.08556,1.63606,2903.76433,1.63228,1504.96639,1.64087,3356.52655
min,0.00043,0.01000,16.35500,16.35000,1.00000,16.36000,1.00000,16.34000,3.00000,16.37000,...,16.43000,1.00000,16.27000,30.00000,16.44000,1.00000,15.24000,2.00000,16.45000,1.00000
25%,0.00048,0.01000,19.01500,19.01000,700.00000,19.02000,700.00000,19.00000,1600.00000,19.03000,...,19.10000,300.00000,18.93000,200.00000,19.12000,200.00000,18.92000,100.00000,19.15000,100.00000
50%,0.00056,0.01000,20.83500,20.83000,1800.00000,20.84000,1800.00000,20.82000,2900.00000,20.85000,...,20.93000,2100.00000,20.75000,400.00000,20.97000,1700.00000,20.74000,200.00000,21.01000,2300.00000
75%,0.00095,0.02000,21.79500,21.79000,3500.00000,21.80000,3600.00000,21.78000,4200.00000,21.81000,...,21.88000,2500.00000,21.71000,900.00000,21.92000,2500.00000,21.70000,500.00000,21.96000,2700.00000
max,0.01985,0.44000,23.46000,23.45000,21611.00000,23.49000,19201.00000,23.44000,25826.00000,23.53000,...,23.93000,16700.00000,23.37000,15700.00000,25.20000,15600.00000,23.36000,15500.00000,26.02000,15600.00000


In [64]:
df_market_l10

,#RIC,Date-Time,rel.spread,abs.spread,mid_price,L1-BidPrice,L1-BidSize,L1-AskPrice,L1-AskSize,L2-BidPrice,...,L8-AskPrice,L8-AskSize,L9-BidPrice,L9-BidSize,L9-AskPrice,L9-AskSize,L10-BidPrice,L10-BidSize,L10-AskPrice,L10-AskSize
36748,SOXS.K,2025-01-03 09:40:00.006514971-05:00,0.00094,0.02000,21.30000,21.29000,800.00000,21.31000,2600.00000,21.28000,...,21.38000,1900.00000,21.21000,100.00000,21.43000,100.00000,21.20000,100.00000,21.44000,2300.00000
36749,SOXS.K,2025-01-03 09:40:00.016467552-05:00,0.00047,0.01000,21.30500,21.30000,600.00000,21.31000,2600.00000,21.29000,...,21.38000,1900.00000,21.22000,100.00000,21.43000,100.00000,21.21000,100.00000,21.44000,2300.00000
36758,SOXS.K,2025-01-03 09:40:00.076256984-05:00,0.00094,0.02000,21.31000,21.30000,1100.00000,21.32000,1300.00000,21.29000,...,21.43000,100.00000,21.22000,100.00000,21.44000,2300.00000,21.21000,100.00000,21.52000,2300.00000
36762,SOXS.K,2025-01-03 09:40:00.081235935-05:00,0.00047,0.01000,21.31500,21.31000,600.00000,21.32000,1300.00000,21.30000,...,21.43000,100.00000,21.23000,2000.00000,21.44000,2300.00000,21.22000,100.00000,21.52000,2300.00000
36777,SOXS.K,2025-01-03 09:40:00.116134193-05:00,0.00094,0.02000,21.31000,21.30000,800.00000,21.32000,3900.00000,21.29000,...,21.43000,100.00000,21.22000,100.00000,21.44000,2300.00000,21.21000,100.00000,21.52000,2300.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15803460,SOXS.K,2025-01-31 15:49:58.506118967-05:00,0.00097,0.02000,20.56000,20.55000,3700.00000,20.57000,1300.00000,20.54000,...,20.69000,2400.00000,20.47000,100.00000,20.77000,2400.00000,20.46000,100.00000,21.03000,100.00000
15803466,SOXS.K,2025-01-31 15:49:58.511100868-05:00,0.00049,0.01000,20.56500,20.56000,600.00000,20.57000,600.00000,20.55000,...,20.69000,2400.00000,20.48000,2500.00000,20.77000,2400.00000,20.47000,100.00000,21.03000,100.00000
15803505,SOXS.K,2025-01-31 15:49:58.941228370-05:00,0.00097,0.02000,20.57000,20.56000,1400.00000,20.58000,800.00000,20.55000,...,20.69000,2400.00000,20.48000,2500.00000,20.77000,2400.00000,20.47000,100.00000,21.03000,100.00000
15803518,SOXS.K,2025-01-31 15:49:59.701817156-05:00,0.00049,0.01000,20.56500,20.56000,700.00000,20.57000,2500.00000,20.55000,...,20.69000,2400.00000,20.48000,2500.00000,20.77000,2400.00000,20.47000,100.00000,21.03000,100.00000


In [65]:
df_market_l10[~df_market_l10.isna().any(axis=1)].to_csv(f"data/{ticker}_cleaned_jan2025.csv", index=False)

In [66]:
df_market_l10.describe()

,rel.spread,abs.spread,mid_price,L1-BidPrice,L1-BidSize,L1-AskPrice,L1-AskSize,L2-BidPrice,L2-BidSize,L2-AskPrice,...,L8-AskPrice,L8-AskSize,L9-BidPrice,L9-BidSize,L9-AskPrice,L9-AskSize,L10-BidPrice,L10-BidSize,L10-AskPrice,L10-AskSize
count,956903.00000,956903.00000,956903.00000,956903.00000,956903.00000,956903.00000,956903.00000,956903.00000,956903.00000,956903.00000,...,956903.00000,956903.00000,956903.00000,956903.00000,956903.00000,956903.00000,956903.00000,956903.00000,956903.00000,956903.00000
mean,0.00074,0.01505,20.41497,20.40745,2399.29856,20.42249,2452.23569,20.39739,3162.02579,20.43262,...,20.50441,2019.21080,20.32711,931.25248,20.53250,2199.89450,20.31702,716.47434,20.56987,2667.51158
std,0.00041,0.00831,1.63234,1.63231,2222.35413,1.63238,2216.75547,1.63237,2073.98572,1.63241,...,1.63401,1929.40852,1.63229,1339.08556,1.63606,2903.76433,1.63228,1504.96639,1.64087,3356.52655
min,0.00043,0.01000,16.35500,16.35000,1.00000,16.36000,1.00000,16.34000,3.00000,16.37000,...,16.43000,1.00000,16.27000,30.00000,16.44000,1.00000,15.24000,2.00000,16.45000,1.00000
25%,0.00048,0.01000,19.01500,19.01000,700.00000,19.02000,700.00000,19.00000,1600.00000,19.03000,...,19.10000,300.00000,18.93000,200.00000,19.12000,200.00000,18.92000,100.00000,19.15000,100.00000
50%,0.00056,0.01000,20.83500,20.83000,1800.00000,20.84000,1800.00000,20.82000,2900.00000,20.85000,...,20.93000,2100.00000,20.75000,400.00000,20.97000,1700.00000,20.74000,200.00000,21.01000,2300.00000
75%,0.00095,0.02000,21.79500,21.79000,3500.00000,21.80000,3600.00000,21.78000,4200.00000,21.81000,...,21.88000,2500.00000,21.71000,900.00000,21.92000,2500.00000,21.70000,500.00000,21.96000,2700.00000
max,0.01985,0.44000,23.46000,23.45000,21611.00000,23.49000,19201.00000,23.44000,25826.00000,23.53000,...,23.93000,16700.00000,23.37000,15700.00000,25.20000,15600.00000,23.36000,15500.00000,26.02000,15600.00000
